Import all the relevant stuff

In [1]:
# eeg analysis libraries
import mne

# import plotting libraries
from matplotlib import pyplot as plt
import seaborn as sns
from plotnine import *

# numerical libraries
import numpy as np
import pandas as pd

# import tqdm for a progress bar:
try:
    from tqdm import tqdm_notebook as tqdm
except:
    from tqdm import tqdm

# import file path operators
from pathlib import Path
import os

# avoid MNE being too verbose
mne.set_log_level('ERROR')

//anaconda/envs/py36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


List all files

In [2]:
datafolder = Path('./Data')

datafiles = list(datafolder.glob(pattern='*RS.bdf'))

group = [f.name[3:6] for f in datafiles]

pids = [f.name[7:12] for f in datafiles]

In [3]:
datafiles

[PosixPath('Data/EG-CTR-0001-RS.bdf'), PosixPath('Data/EG-CTR-0025-RS.bdf')]

Run preprocessing

In [46]:
for file in tqdm(datafiles):
    
    # create an MNE raw structure
    raw = mne.io.read_raw_edf(file, montage=mne.channels.read_montage('biosemi64'),eog=[f'EXG{n}' for n in range(1, 9)])
    
    raw.info['subject_info'] = {'pid': file.name[7:11], 'group': file.name[3:6], 'subject_id': file.name[0:11]}
    
    # find the trigger codes (this needs to be checked somehow to make sure we get the correct timings as 
    # they were somewhat messed up...)
    events = mne.find_events(raw)
    
    # segment the data only take the second and fourth event (add in some check for the length of each epoch 
    # before selecting them)
    epochs = [mne.Epochs(raw, events[[1,3],:], tmin=0, tmax=45, 
                         baseline=(None, 0), picks=None, 
                         name=None, preload=True, reject=None, 
                         flat=None, proj=True, decim=1, 
                         reject_tmin=None, reject_tmax=None, 
                         detrend=None, add_eeg_ref='EXG6', 
                         on_missing='error', reject_by_annotation=True, 
                         verbose=None)]
    
    # do some filtering
    #raw = raw.resample(256)
    #raw = raw.filter(50,48)
    
    


RuntimeError: raw.resample requires raw data to be loaded. Use preload=True (or string) in the constructor or raw.load_data().

In [43]:
raw.ch_names

['Fp1',
 'AF7',
 'AF3',
 'F1',
 'F3',
 'F5',
 'F7',
 'FT7',
 'FC5',
 'FC3',
 'FC1',
 'C1',
 'C3',
 'C5',
 'T7',
 'TP7',
 'CP5',
 'CP3',
 'CP1',
 'P1',
 'P3',
 'P5',
 'P7',
 'P9',
 'PO7',
 'PO3',
 'O1',
 'Iz',
 'Oz',
 'POz',
 'Pz',
 'CPz',
 'Fpz',
 'Fp2',
 'AF8',
 'AF4',
 'AFz',
 'Fz',
 'F2',
 'F4',
 'F6',
 'F8',
 'FT8',
 'FC6',
 'FC4',
 'FC2',
 'FCz',
 'Cz',
 'C2',
 'C4',
 'C6',
 'T8',
 'TP8',
 'CP6',
 'CP4',
 'CP2',
 'P2',
 'P4',
 'P6',
 'P8',
 'P10',
 'PO8',
 'PO4',
 'O2',
 'EXG1',
 'EXG2',
 'EXG3',
 'EXG4',
 'EXG5',
 'EXG6',
 'EXG7',
 'EXG8',
 'STI 014']

In [ ]:
## copied from Jans MEG analysis, needs to be adapted :)
    # crop
    raw = raw.crop(tmin=20, tmax=500)
    # resample
    raw = raw.load_data()
    raw = raw.resample(256)
    # filter
    # filter the MEG data (exclude line noise)
    raw = raw.filter(0.5, 30, picks=mne.pick_types(raw.info, meg=True))
    # filter the EOG data
    raw = raw.filter(0.5, 15, picks=mne.pick_types(raw.info, meg=False, eog=True))
    # filter the ECG data
    raw = raw.filter(0.5, 15, picks=mne.pick_types(raw.info, meg=False, ecg=True))
    # maxwell-correction
    raw = mne.preprocessing.maxwell_filter(raw, cross_talk=str(ctfile), calibration=str(ssscal),
                                           st_duration=10, st_correlation=0.98)
    # pick gradiometers
    picks = mne.pick_types(raw.info, meg='grad', eeg=False, stim=False, eog=False, exclude='bads')
    # run an ICA
    try:
        ica = mne.preprocessing.run_ica(raw, n_components=0.95,
                                        picks=picks,
                                        eog_ch=veog, ecg_ch=ecg)
        raw = ica.apply(raw, exclude=ica.exclude)
    except RuntimeError:
        continue

Run coherence analysis on the output

In [ ]:


mne.connectivity.spectral_connectivity(data, method=['wpli', 'wpli2_debiased'], 
                                       indices=None, sfreq=6.283185307179586, 
                                       mode='multitaper', fmin=None, fmax=inf, 
                                       fskip=0, faverage=False, tmin=None, tmax=None, 
                                       mt_bandwidth=None, mt_adaptive=False, mt_low_bias=True, 
                                       cwt_frequencies=None, cwt_n_cycles=7, block_size=1000, 
                                       n_jobs=1, verbose=None)